In [28]:
import numpy as np
import CoolProp.CoolProp as CP


### Specify Parameters

In [29]:
# tsp for thermosyphon
l_e= 0.51 # evaporator tsp len
l_a=0.2 # evaporator tsp len
l_c=0.51 # condenserr tsp len
tsp_od=9.55/1000
tsp_thick=0.89/1000
inlet_dia=60/1000
m = 3.8/60 # 3.8 L/min to kg/s
k_ss=16.2 # W/mK


### Calculate Primary parameters

In [30]:
l_ef=l_e/2+l_a+l_c/2 # effective length

### Evaporator

In [31]:
T_inf_e=100 #°C
T_s_e=70 #°C
T_f=(T_s_e+T_inf_e)/2

fluid = "Water"
temperature = T_f + 273.15  # Convert °C to Kelvin
pressure = 101325  # Pressure in Pa (1 atm)

#properties
density = CP.PropsSI('D', 'T', temperature, 'P', pressure, fluid)  # Density in kg/m³
viscosity = CP.PropsSI('V', 'T', temperature, 'P', pressure, fluid)  # Dynamic viscosity (Pa·s)
C = CP.PropsSI('C', 'T', temperature, 'P', pressure, fluid)  # Specific heat capacity (J/(kg·K))
k = CP.PropsSI('L', 'T', temperature, 'P', pressure, fluid)  # Thermal conductivity (W/(m·K))
Pr=CP.PropsSI('PRANDTL', 'T', temperature, 'P', pressure, fluid) # Prandtl Number

print(f"Properites of {fluid} at {T_f} °C:")
print(f"Density: {density} kg/m³")
print(f"Viscosity: {viscosity} Pa·s")
print(f"Prandtl Number: {Pr}")
print(f"Thermal Conductivity: {k}")



Properites of Water at 85.0 °C:
Density: 968.6114401082684 kg/m³
Viscosity: 0.0003330754563272273 Pa·s
Prandtl Number: 2.088096210424027
Thermal Conductivity: 0.6700671488300696


In [32]:
u=m/(density*np.pi/4*inlet_dia**2)
print(f"Hot Water Velocity: {u}")

Re=density*u*tsp_od/viscosity
print(f"Renolds Number: {Re}")

# Churchil and Bernstein
Nu = 0.3 + 0.62*pow(Re,0.5)*pow(Pr,1/3)/pow(1+pow(0.4/Pr,2/3),1/4)*pow(1+pow(Re/282000,5/8),4/5)
print(f"Nusselt Number: {Nu}")

#coefficient of external heat transer in evaporator
h_ex_e=Nu*k/tsp_od
print(f"heat transfer coefficient, h_ex_e: {h_ex_e}")

A_e=np.pi*tsp_od*l_e
# Convection resistance in hot fluid
R_ex_hs=1/(h_ex_e*A_e)
print(f"R_ex_hs: {R_ex_hs}")

r_e=tsp_od/2
r_i=r_e-tsp_thick
R_cond_e=np.log(r_e/r_i)/(2*np.pi*k_ss*l_e)
print(f"R_cond: {R_cond_e}")


Hot Water Velocity: 0.02312545945234761
Renolds Number: 642.2449589332427
Nusselt Number: 19.325624934451945
heat transfer coefficient, h_ex_e: 1355.96506797775
R_ex_hs: 0.048197895765966316
R_cond: 0.003973498873512054


### Condenser

In [33]:
T_inf_c=20 #°C
T_s_c=40 #°C ASSUME
T_f=(T_s_c+T_inf_c)/2

fluid = "Water"
temperature = T_f + 273.15  # Convert °C to Kelvin
pressure = 101325  # Pressure in Pa (1 atm)

#properties
density = CP.PropsSI('D', 'T', temperature, 'P', pressure, fluid)  # Density in kg/m³
viscosity = CP.PropsSI('V', 'T', temperature, 'P', pressure, fluid)  # Dynamic viscosity (Pa·s)
C = CP.PropsSI('C', 'T', temperature, 'P', pressure, fluid)  # Specific heat capacity (J/(kg·K))
k = CP.PropsSI('L', 'T', temperature, 'P', pressure, fluid)  # Thermal conductivity (W/(m·K))
Pr=CP.PropsSI('PRANDTL', 'T', temperature, 'P', pressure, fluid) # Prandtl Number

print(f"Properites of {fluid} at {T_f} °C:")
print(f"Density: {density} kg/m³")
print(f"Viscosity: {viscosity} Pa·s")
print(f"Prandtl Number: {Pr}")
print(f"Thermal Conductivity: {k}")



Properites of Water at 30.0 °C:
Density: 995.6494539376417 kg/m³
Viscosity: 0.0007972217998101543 Pa·s
Prandtl Number: 5.4236420311135705
Thermal Conductivity: 0.6143922004176029


In [34]:
u=m/(density*np.pi/4*inlet_dia**2)
print(f"Cold Water Velocity: {u}")

Re=density*u*tsp_od/viscosity
print(f"Renolds Number: {Re}")

# Churchil and Bernstein
Nu = 0.3 + 0.62*pow(Re,0.5)*pow(Pr,1/3)/pow(1+pow(0.4/Pr,2/3),1/4)*pow(1+pow(Re/282000,5/8),4/5)
print(f"Nusselt Number: {Nu}")

#coefficient of external heat transer in condenser
h_ex_c=Nu*k/tsp_od
print(f"heat transfer coefficient, h_ex_c: {h_ex_c}")

A_c=np.pi*tsp_od*l_c
# Convection resistance in cold fluid
R_ex_cs=1/(h_ex_e*A_c)
print(f"R_ex_hs: {R_ex_cs}")

r_e=tsp_od/2
r_i=r_e-tsp_thick
R_cond_c=np.log(r_e/r_i)/(2*np.pi*k_ss*l_e)
print(f"R_cond: {R_cond_c}")


Cold Water Velocity: 0.022497460822899912
Renolds Number: 268.3268731756857
Nusselt Number: 17.612537295725822
heat transfer coefficient, h_ex_c: 1133.0895857652445
R_ex_hs: 0.048197895765966316
R_cond: 0.003973498873512054


In [53]:
# Overal Resistance
R_o=R_ex_hs+R_cond_e+R_cond_c+R_ex_cs

q=(T_s_e-T_s_c)/R_o
print(f"Based on assumption, q: {q}")


Based on assumption, q: 287.5138781252633


### Thermal Resistance of Evaporator Pool

In [ ]:
# Pool Parameter
Chi